## Normal-continuous HDG

In the following we consider an $H(div)$-conforming approach to discretize the Stokes equations. 
This has some advantages like:
* a simple convection stabilization via an up-winding approach when including the non-linear transport of the Navier-Stokes equations and
* an exact conservation of mass (and several other properties such as pressure-robustness, energy-stability for Navier-Stokes,...) 

For a triangulation $\mathcal{T}_h$ with its set of facets (the skeleton 🦴 ☠️) $\mathcal{F}_h$ we consider the spaces 

\begin{align*}
V^T_h &= \{u_h \in H(\operatorname{div}, \Omega): u_h|_T \in [\mathbb{P}(T)]^d ~ \forall T \in \mathcal{T}_h \}, \\
\hat V_h &= \{\hat u_h \in L^2(\mathcal{F}_h): \hat u_h|_F \in [\mathbb{P}^k(F)]^d, \hat u_h|_F \cdot n = 0 \, \forall F \in \mathcal{F}_h \},
\end{align*}

and set $V_h = V_h^T \times \hat V_h$ and $Q_h = \mathbb{P}^{k-1}$. Note, whereas functions in $V_h^T$ have a continuous normal component, discrete pressure functions are **not** continuous. Further, functions in $\hat V_h$ represent vector valued polynomials that lie in the tangential plane of the facets.

```{note}
Considering simplices, $V_h^T$ is the classical Brezzi-Douglas-Marini space of order $k$.
```

```{note}
For details on the discretization we refer to "High order exactly divergence-free Hybrid Discontinuous Galerkin Methods for unsteady incompressible flows, C. Lehrenfeld, J. Schöberl, Computer Methods in Applied Mechanics and Engineering, https://doi.org/10.1016/j.cma.2016.04.025".
```

We consider the problem: Find $(u_h, \hat u_h) \in V_h$ and $p_h \in Q_h$ with $u_h \cdot n = u_D \cdot n $ and $\hat u_h = (u_D)_t$ such that

\begin{align}
% K((u_h, \hat u_h) ,p ; (v_h, \hat v_h), q) = 
a_h((u_h, \hat u_h),(v_h, \hat v_h)) &- b(v_h,p_h) -b(u_h,q_h) = (f,v_h), \tag{S}
\\
&\forall (v_h, \hat v_h) \in V_h, ~ v_h \cdot n = 0, \hat v_h = 0 \text{ on } \Gamma_{in} \cup \Gamma_{wall},
\quad
\forall q_h \in Q_h, \nonumber
\end{align}

where for $\gamma > 0$ sufficiently large, we have the symmetric interior-penalty bilinear form 

\begin{align*}
a_h((u_h, \hat u_h),(v_h, \hat v_h)) = &
\sum_{T \in \mathcal{T}_h} \nu \int_T \nabla u_h: \nabla v_h 
- \int_{\partial T} \nu \nabla u_h n \cdot (v_h - \hat v_h)_t\\
&- \int_{\partial T} \nu \nabla v_h n \cdot (u_h - \hat u_h)_t 
+ \frac{\nu \gamma k^2}{h}\int_{\partial T} (u_h - \hat u_h)_t \cdot (v_h - \hat v_h)_t,
\end{align*}

with the tangential projection $ (\phi)_t = \phi - (\phi \cdot n) n$ for a function $\phi$.

Note, that Dirichlet (inlet, slip,...) boundary conditions are split into their normal and tangential components. This can be particularly helpful if the considered boundary is not parallel to the coordinate axis. 

Further since 

$$
\operatorname{div} V_h^T \subset Q_h,
$$

with $b(v_h,p_h) = \int_{\Omega} \operatorname{div} v_h p_h$ we can choose $q_h = \operatorname{div} u_h$ in $(S)$ and we obtain $\Vert \operatorname{div} u_h \Vert_{\Omega} = 0$ and thus the discrete velocity solution is **exactly divergence-free** pointwise. 

### A simple example with slip-boundary conditions

We consider a Stokes flow on a cylinder $\Omega = \{(x,y,z) \in \mathbb{R}^d: \| (x,y)\|_2 < 1, 0 < 0 < 0.5 \}$ with a non-zero right hand side $f = 25 (0,0, \| (x,y)\|_2 - 1/2)$, and boundary conditions

\begin{align*}
u = 0 \quad &\text{on} \quad \Gamma_{top} = \{(x,y,z) \in \mathbb{R}^d: \| (x,y)\|_2 \le 1, z = 0.5 \}, \\
u\cdot n = 0, u_t = (y,-x,0) \quad &\text{on} \quad \Gamma_{bottom} = \{(x,y,z) \in \mathbb{R}^d: \| (x,y)\|_2 \le 1, z = 0.0 \}, \\
u\cdot n = 0 \quad &\text{on} \quad \Gamma_{side} = \{(x,y,z) \in \mathbb{R}^d: \| (x,y)\|_2 \le 1, z = 0.0 \}.
\end{align*}

In [ ]:
from ngsolve import *
from netgen.occ import *


cyl = Cylinder((0,0,0), Z, h=0.5,r=1)
cyl.faces[0].name="side"
cyl.faces[1].name="top"
cyl.faces[2].name="bottom"

ngmesh = OCCGeometry(cyl).GenerateMesh(maxh=0.5)
mesh = Mesh(ngmesh)
mesh.Curve(3)
from ngsolve.webgui import Draw
Draw(mesh)

We choose a viscosity $\nu = 1$, polynomials of order 3 (for the velocity) and a stabilization parameter of $\gamma = 10$. Further, since we now have a problem with a prescribed Dirichlet value (for the normal component) on the whole boundary we consider the pressure in $L^2_0(\Omega)$ which we enforce via an additional constraint using the ```NumberSpace```, i.e. 

$$
\int_\Omega p_h \mu_h = 0 \quad \mu_h \in \mathbb{R},
$$

and the corresponding term for the Lagrange multiplier. Further, we want to use a static condensation for the high-order velocity and pressure functions. To make sure that we can solve for the Schur complement we need to mark the lowest order pressures (i.e. the constants) such that they are not condensed. While this can be done in many various ways (see [NGSolve docu - static condensation](https://docu.ngsolve.org/nightly/i-tutorials/unit-1.4-staticcond/staticcond.html)), we simply add the ```lowest_order_wb=True``` flag to the pressure space.

In [ ]:

nu = 1
order = 3
gamma = 10

VT = HDiv(mesh,order=order, dirichlet="top|bottom|side")
Vhat = TangentialFacetFESpace(mesh,order=order, dirichlet="bottom|top")
Q = L2(mesh,order=order-1, lowest_order_wb=True)
N = NumberSpace(mesh)

X = VT * Vhat * Q * N

(u, uhat, p, lam), (v, vhat, q, mu) = X.TnT()

n = specialcf.normal(mesh.dim)
h = specialcf.mesh_size

def tang(u):
    return u - (u*n)*n

In [ ]:

K = BilinearForm(X, symmetric=True, condense = True)
K += (nu*InnerProduct(Grad(u), Grad(v)) + div(u)*q + div(v)*p + p * mu + q * lam) * dx()
K += nu * Grad(u)*n * tang(vhat - v) * dx(element_boundary = True)
K += nu * Grad(v) * n *  tang(uhat-u) * dx(element_boundary = True)
K += nu  *gamma*order**2/h * InnerProduct ( tang(vhat-v),  tang(uhat-u) ) * dx(element_boundary = True)
K.Assemble()


In [ ]:
gfu_bc = GridFunction(X)
gfu_bc.components[1].Set(CF((y,-x,0)), definedon=mesh.Boundaries("bottom"))

In [ ]:
KSinv = K.mat.Inverse(X.FreeDofs(coupling=True))
Kinv = ((IdentityMatrix(K.mat.height) + K.harmonic_extension) @ (KSinv + K.inner_solve) @ (IdentityMatrix(K.mat.height) + K.harmonic_extension_trans))

Let's start the problem without external forces:

In [ ]:
gfu = GridFunction(X)
rhs = gfu.vec.CreateVector()
rhs.data = -K.mat * gfu_bc.vec
gfu.vec.data = gfu_bc.vec + Kinv * rhs

Visualize the solution

In [ ]:
from ngsolve.webgui import FieldLines

N = 12
points = [ (sin(2*pi*k/N)*i/N, cos(2*pi*k/N)*i/N, j/N)   for i in range(1,N) for j in range(1,N) for k in range(0,N)]

fl = FieldLines(gfu.components[0], mesh.Materials('.*'), num_lines=N**3//20, length=1)

ea = { "euler_angles" : (-40, 0, 0) }

Draw(gfu.components[0], mesh,  "X", draw_vol=False, draw_surf=True, objects=[fl], \
     min=0, max=1, autoscale=False, settings={"Objects": {"Surface": False, "Wireframe":False}}, **ea);

Now, we additionally define a body force $f = 25 (0,0, \| (x,y)\|_2 - 1/2)$ to the right hand side. The force is showing downwards in the center and upwards at the boundary. 

In [ ]:
r = sqrt(x**2+y**2)
F = LinearForm(X)
F += 500*CF((0,0,r-0.5))*v*dx
F.Assemble()

In [ ]:
from ngsolve.webgui import AddFieldLines

gfu_vis = GridFunction(X,multidim=True)
steps=10
for i,(val_bc, val_force) in enumerate([(1-j/(steps-1),j/(steps-1)) for j in range(steps)]):
    rhs.data = val_force * F.vec.data - val_bc * K.mat * gfu_bc.vec
    gfu.vec.data = val_bc * gfu_bc.vec.data + Kinv * rhs
    fl = FieldLines(gfu.components[0], mesh.Materials('.*'), num_lines=N**3//20, length=1)
    if i == 0:
        gfu_vis.vec.data = gfu.vec
        fieldlines = fl
    else:
        gfu_vis.AddMultiDimComponent(gfu.vec)
        AddFieldLines(fieldlines, fl) 

Draw(gfu_vis.components[0], mesh,  "X", draw_vol=True, draw_surf=True, objects=[fieldlines], \
     min=0, max=1, autoscale=False, animate=True, settings={"Objects": {"Surface": False, "Wireframe":False}}, **ea);     

### An iterative parallel solver

Since the above chosen discrete velocity space is not conforming with respect to the $[H^1]^d$ space we cannot use the preconditioners for the $A$-block discussed so far. To fix this we make use of an auxiliary space preconditioner where an approximate to the inverse of $A$ is given by 

$$
\hat A^{-1} = A^{-1}_{smooth} + E \hat A_{cg}^{-1} E^T,
$$

Here, $A_{smooth}$ is some appropriate smoother, e.g. some (block) Jacobi-preconditioner, $\hat A_{cg}$ is an available preconditioner for a (low-order) vector-valued Laplace operator discretized using a **continuous** finite element space $V_h^{cg} \subset V_h$ and $E$ is the FEM matrix of the embedding operator 

$$
e: V_h^{cg} \rightarrow V_h. 
$$

*Remark: More details on the $H(\operatorname{div})$-conforming preconditioner can be found in "A conforming auxiliary space preconditioner for the mass conserving stress-yielding method, L. Kogler, P.L. Lederer, J. Schöberl, Numerical Linear Algebra with Applications, https://doi.org/10.1002/nla.2503"*

Depending on how big your system is (available ```MPI_RANKS```), you can choose between solving again:

- the flow over the NACA2412 airfoil ([naca_geometry.py](naca_geometry.py)).
- the flow around an airplane (geometry available via ```OCCGeometry``` and the step file [plane.step](plane.step)).

In [ ]:
# predefined flags, number of ranks and available memory
from commonCFD import *
from ipyparallel import Cluster
c = await Cluster(engines="mpi").start_and_connect(n=MPI_RANKS, activate=True)

In [ ]:
%%px
from commonCFD import *
from mpi4py import MPI
from ngsolve import *
from netgen.occ import *
# from ngsolve.krylovspace import CGSolver
from ngsolve.krylovspace import MinResSolver, GMResSolver, BramblePasciakCG
ngsglobals.numthreads=1

import ngsolve.ngs2petsc as n2p
import petsc4py.PETSc as psc

from naca_geometry import *

if MPI_RANKS <= 4:
    DIM = 2
    geom = OCCGeometry(occ_naca_profile(type = "2412", depth=0 if DIM==2 else 2, height=4, angle=4, h=0.05), dim=DIM)
else:
    geom = OCCGeometry("plane.step")

if (MPI.COMM_WORLD.rank == 0):
    if MPI_RANKS <= 4:
        ngmesh = geom.GenerateMesh(maxh=0.2,  grading=0.9)
    else:
        ngmesh = geom.GenerateMesh(maxh=2)
        for ind in range(1, 1+ngmesh.GetNFaceDescriptors()):
            ngmesh.SetBCName(ind-1, "wall")
    
        for ind, name in [ (4, "bottom"),
                            (2, "top"),
                            (5, "right"), # front of plane - inflow
                            (1, "inlet"), # front of plane - inflow
                            (3, "left"), # front of plane - inflow
                            (6, "back")]: # back of plane - outflow
            ngmesh.SetBCName(ind-1, name)
    # Mesh(ngmesh)
    ngmesh = ngmesh.Distribute(MPI.COMM_WORLD)
else:
    ngmesh = netgen.meshing.Mesh.Receive(MPI.COMM_WORLD)

mesh = Mesh(ngmesh)
# mesh.ngmesh.SetGeometry(geom)
# mesh.Curve(3)
nu = 1e-4

We eventually want to use the ```BSPC``` preconditioner and a ```GMRes``` solver. Thus, we again want to use a ```BlockMatrix``` and ```BlockVector```. Note however, that due to using an HDG method, the velocity space is a product space, i.e. ```V = VT * Vhat```. Further, to make the computations more efficient, we are going to condense the local velocity dofs in the $A$-block. To be able to perform the ```GMRes``` iteration on the full system we need to have access to the full system (and not just the Schur-complement), thus we add the flag ```store_inner=True```.

In [ ]:
%%px
if MPI_RANKS <= 4:
    order = 2
else:
    order = 1
    
gamma = 10

VT = HDiv(mesh,order=order, dirichlet="wall|inlet")
Vhat = TangentialFacetFESpace(mesh,order=order, dirichlet="wall|inlet")
Q = L2(mesh,order=order-1)

V = VT * Vhat

(u, uhat), (v,vhat) = V.TnT()
p,q = Q.TnT()

n = specialcf.normal(mesh.dim)
h = specialcf.mesh_size

def tang(u):
    return u - (u*n)*n

elint = True

bfa = BilinearForm(V, condense = elint, store_inner = elint) #, symmetric=True)
bfa += (nu*InnerProduct(Grad(u), Grad(v))) * dx()
bfa += nu * Grad(u)*n * tang(vhat - v) * dx(element_boundary = True)
bfa += nu * Grad(v) * n *  tang(uhat-u) * dx(element_boundary = True)
bfa += nu  *gamma*order*(order+1)/h * InnerProduct ( tang(vhat-v),  tang(uhat-u) ) * dx(element_boundary = True)
bfa.Assemble()
A = bfa.mat

bfb = BilinearForm(div(u)*q*dx).Assemble()
B = bfb.mat

F = LinearForm(V).Assemble()
G = LinearForm(Q).Assemble()

rhs_vec = BlockVector([F.vec, G.vec])

As a next step we want to setup the auxiliary space ```Vc``` and the corresponding solver. This follows very similar steps as in the previous section. Note however, that we only consider a lowest order linear space here.

In [ ]:
%%px

Vc = VectorH1(mesh, order = 1, dirichlet = "inlet|wall")
V1c = H1(mesh, order=1, dirichlet="inlet|wall")

u1c, v1c = V1c.TnT()
bfa1c = BilinearForm(nu * InnerProduct(Grad(u1c),Grad(v1c))*dx)

hata1c = Preconditioner(bfa1c, "gamg")
bfa1c.Assemble()

hatAcinv = sum( [Ei@hata1c@Ei.T for Ei in Vc.embeddings])

To be able to use the solver for our discretization space we set up the aforementioned embedding. In NGSolve this can be realized using a ```ConvertOperator``` that uses the functionals (degrees of freedoms) of the ```spaceb``` to interpolate (convert) a function from ```spacea```. By changing the ```rangedofs``` one can control the actual considered functionals (e.g. the range that is associated to the corresponding basis functions). 

Note, that in our case the velocity space ```V``` is a product space of ```VT``` and ```Vhat```, thus we have to setup the embedding for each component separately. Using the embeddings of the product space ```V``` we can eventually define the overall operator.

In [ ]:
%%px
emb1 = ConvertOperator(spacea = Vc, spaceb = VT, localop = True, range_dofs = VT.FreeDofs(elint))
tc1 = V.Embedding(0)

emb2 = ConvertOperator(spacea = Vc, spaceb = Vhat, localop = True, range_dofs = Vhat.FreeDofs(elint))
tc2 = V.Embedding(1)

embA = tc1 @ emb1 + tc2 @ emb2

\begin{align*}
E_{\texttt{Vc}\rightarrow\texttt{V}}
= 
\begin{pmatrix}
E_{\texttt{Vc}\rightarrow\texttt{VT}} \\
E_{\texttt{Vc}\rightarrow\texttt{Vhat}} 
\end{pmatrix}
= 
\begin{pmatrix}
E_{\texttt{Vc}\rightarrow\texttt{VT}} \\
0
\end{pmatrix}
+
\begin{pmatrix}
0 \\
E_{\texttt{Vc}\rightarrow\texttt{Vhat}} 
\end{pmatrix}
=
\begin{pmatrix}
I_{\texttt{VT}} \\
0
\end{pmatrix} \cdot 
E_{\texttt{Vc}\rightarrow\texttt{VT}}
+
\begin{pmatrix}
0 \\
I_{\texttt{Vhat}} 
\end{pmatrix} \cdot
E_{\texttt{Vc}\rightarrow\texttt{Vhat}} 
\end{align*}

The final operator needed is a block smoother. Since we are considering the statically condensed system, the Schur complement only includes degrees of freedoms (dofs) associated to facets. To this end we aim for a block Jacobi smoother where each block includes all dofs (both of ```V``` and ```Vhat```) of one facet. This can be easily done using the ```CreateSmoothingBlocks``` function.

Note that this also helps when considering the MPI parallel implementation since shared dofs are only on facets.  

In [ ]:
%%px
sm_blocks = V.CreateSmoothingBlocks(blocktype=["facet"], condense = elint)
bsmoother = bfa.mat.local_mat.CreateBlockSmoother(blocks = sm_blocks, parallel=True)

hatAinv = embA @ hatAcinv @ embA.T + bsmoother 

\begin{align*}
  \hat{A}^{-1} = B_{\text{smoother}}^{-1} + E_{\texttt{Vc}\rightarrow\texttt{V}} \hat{A}_{\texttt{V}}^{-1} E_{\texttt{Vc}\rightarrow\texttt{V}}^T
\end{align*}


Before we can eventually define our ```BlockMatrix``` for the solver and the system, note, that we want to perform the iteration on the full system. Thus, we need to include the remaining inner dofs using the matrices provided by NGSolve. Since these matrices will result in the forward operator ```AA_par``` which is an operation from a consistent to distributed state, ```ParallelMatrix.C2D```, and a backward operator ```hatAAinv``` which is ```ParallelMatrix.D2C``` we have to be careful when adding up these matrices. 

In [ ]:
%%px
Ahex, Ahext, Aiii, Aii  = bfa.harmonic_extension.local_mat, bfa.harmonic_extension_trans.local_mat, bfa.inner_solve.local_mat, bfa.inner_matrix.local_mat
Id = IdentityMatrix(A.height)

# parallel version of Schurcomplement strategy:
Ihex = ParallelMatrix(Id + Ahex, row_pardofs = A.row_pardofs, col_pardofs = A.row_pardofs, op = ParallelMatrix.C2C)
Ihext = ParallelMatrix(Id + Ahext, row_pardofs = A.row_pardofs, col_pardofs = A.row_pardofs, op = ParallelMatrix.D2D)
Isolve = ParallelMatrix(Aiii, row_pardofs = A.row_pardofs, col_pardofs = A.row_pardofs, op = ParallelMatrix.D2C)
hatAAinv = ( Ihex @ hatAinv @ Ihext ) + Isolve

AA = (Id - Ahext) @ (A.local_mat + Aii) @ (Id - Ahex)
AA_par = ParallelMatrix(AA, row_pardofs = A.row_pardofs, col_pardofs = A.col_pardofs, op = ParallelMatrix.C2D)

K = BlockMatrix([[AA_par,B.T], [B, None]])

For the pressure block sovler we again choose the mass matrix for the pressure. Since ```Q``` is an ```L2``` space which uses an $L^2$-orthogonal Legendre basis, no mass lumping is needed.

In [ ]:
%%px
hatS = BilinearForm(1/nu * p*q*dx).Assemble()
hatSinv = hatS.mat.Inverse(Q.FreeDofs())

In [ ]:
%%px
hatK_block_inv = BlockMatrix([[hatAinv, None],[None, hatSinv]])

In [ ]:
%%px
gfu = GridFunction(V)
gfp = GridFunction(Q)

if mesh.dim == 3:
    uin = CF((1,0,0))
else:
    uin = CF((1,0))

gfu.components[0].Set(uin, definedon = mesh.Boundaries("inlet"))

sol_vec = BlockVector([gfu.vec, gfp.vec])

rhs_vec.data = -K* sol_vec

In [ ]:
%%px 
from ngsolve import BaseMatrix

class BSPC(BaseMatrix):
    def __init__(self, M, Ahat, Shat):
        super(BSPC, self).__init__()
        self.M = M
        self.A, self.B, self.BT = M[0,0], M[1,0], M[0,1]
        self.Ahat = Ahat
        self.Shat = Shat
        self.mBTSg2 = self.A.CreateColVector()
        self.g2 = self.Shat.CreateColVector()
        self.xtemp = self.M.CreateColVector()
    def IsComplex(self):
        return False
    def Height(self):
        return self.M.height
    def Width(self):
        return self.M.width
    def CreateColVector(self):
        return self.M.CreateColVector()
    def CreateRowVector(self):
        return self.M.CreateRowVector()
    def Mult(self, b, x):
        f, g = b[0], b[1]
        x[0].data = self.Ahat * f
        self.g2.data = g - self.B * x[0]
        x[1].data = - self.Shat * self.g2
        self.mBTSg2.data = self.BT * x[1]
        x[0].data -= self.Ahat * self.mBTSg2
    def MultTrans(self, b, x):
        self.Mult(b, x)
    def MultAdd(self, scal, b, x):
        self.Mult(b, self.xtemp)
        x.data += scal * self.xtemp
    def MultTransAdd(self, scal, b, x):
        self.MultAdd(scal, b, x)

hatK_sz_inv = BSPC(M = K, Ahat = hatAAinv, Shat = hatSinv)

In [ ]:
%%px 
solver = GMResSolver(mat = K, pre = hatK_sz_inv , maxiter=500, \
                     printrates='\r' if mesh.comm.rank==0 else False, tol=1e-6)
sol_vec.data += solver * rhs_vec

In [ ]:
%%px 
vel1 = GridFunction(Vc)
vel1.Set(gfu.components[0])

In [ ]:
with TaskManager():
    gfu = c[:]["vel1"]

In [ ]:
mesh = gfu[0].space.mesh

if MPI_RANKS <= 4:
    N = 12 
    fac = 0 if mesh.dim == 2 else 1
    p = [(-1+4*i/N,-2+4*j/N,fac * 2*k/N) for i in range(1,2*N) for j in range(1,2*N) for k in range(1,N)]
    clipping = { "clipping" : { "y":0, "z":-1} }

    with TaskManager():
        fieldlines = gfu[0]._BuildFieldLines(mesh, p, num_fieldlines=N**3//5, randomized=True, length=0.3)    
        Draw(gfu[0], mesh,  "X", draw_vol=False, draw_surf=True, objects=[fieldlines], \
            min=0, max=1, autoscale=False, settings={"Objects": {"Surface": False}}, **clipping);
else:
    clipping = { "clipping" : { "y":1, "z":0} }
    with TaskManager():
        N = 4
        p = [(-6,-4+4*j/(2*N),-0.8 + 0.8*k/N) for j in range(1,2*N) for k in range(1,N)]
        fieldlines = gfu[0]._BuildFieldLines(mesh, p, num_fieldlines=int(N**3//3.5), randomized=False, length=1, direction = 1)    
        p2 = [(-6,4 - 4*j/(2*N),-0.8 + 0.8*k/N) for j in range(1,2*N) for k in range(1,N)]
        fieldlines2 = gfu[0]._BuildFieldLines(mesh, p2, num_fieldlines=int(N**3//3.5), randomized=False, length=1, direction = 1)       
        Draw(gfu[0][0], mesh,  "X", draw_vol=False, draw_surf=True, objects=[fieldlines, fieldlines2], \
            min=0, max=1, autoscale=False, settings={"Objects": {"Surface": False, "Wireframe": False}}, euler_angles=[0,0,-90]);

In [ ]:
c.shutdown(hub=True)